# 2 Собираем датасет транзакций

## Состав сета

Сет транзакций будет состоять из:
- исходного сета с транзакциями клиентов
- разметки гендера по транзакциям 
- справочника MCC
- справочника типов транзакций

Для начала обработаем справочники, так как в них может содержаться информация, которую можно превратить в полезные фичи.

## Выделяем фичи справочников

### Справочник MCC

In [1]:
import pandas as pd
pd.set_option('max_colwidth', 400)

In [2]:
tr_mcc_codes = pd.read_csv('kaggle_datasets/tr_mcc_codes.csv', delimiter=';')
tr_mcc_codes.sample(7, random_state=7)

,mcc_code,mcc_description
182,9399,"Правительственные услуги, нигде ранее не классифицируемые"
179,9211,"Судовые выплаты, включая алименты и детскую поддержку"
131,7011,"Отели, мотели, базы отдыха, сервисы бронирования"
24,4900,"Коммунальные услуги — электричество, газ, санитария, вода"
65,5541,Станции техобслуживания
104,5947,"Магазины открыток, подарков, новинок и сувениров"
32,5074,Оборудование для водопровода и отопительной системы


**Что мы видим**

1. **Ключом справочника является [MCC код](https://ru.wikipedia.org/wiki/Merchant_Category_Code)**. Это международный справочник, в котором перечислены значения отраслей и категорий точек продаж. Последние две цифры кода - категория, а первые 1-2 символа - отрасль.

2.  **Описание категории сложносоставное.** Необходимо привести его упростить.

Выделим из названия основу (до знаков запятой, тире и слэша) и отдельно выделим макроотрасль MCC (первые две цифиы ).

In [3]:
# сохраним текущее значение описания в отдельном поле
tr_mcc_codes['mcc_full_description'] = tr_mcc_codes['mcc_description']


# сформируем описание МСС для дальнейшей работы с ним

## подготовим функцию, выделяющую первую часть значения (до разделителя)
def extract_text_before_delimiter(value):
    delimiters = [',', '—', '/']
    for delimiter in delimiters:
        if delimiter in value:
            value = value.split(delimiter)[0].strip()
    return value

## пройдемся по всем строкам функцией, выделяющей текст до разделителя
for index, row in tr_mcc_codes.iterrows():
    tr_mcc_codes.at[index, 'mcc_description'] = extract_text_before_delimiter(row['mcc_description'])

## добавим к сокращенному описанию значение кода MCC, чтобы исключить дубликаты в этом поле
tr_mcc_codes['mcc_description'] = tr_mcc_codes['mcc_description'] + ' (' + tr_mcc_codes['mcc_code'].astype('str') + ')' 


# выделяем группу MCC отдельной фичей
tr_mcc_codes['mcc_group'] = tr_mcc_codes['mcc_code'].astype('str').apply(lambda x: x[:2])


# сохраним получившийся справочник отдельным файлом
import os
os.makedirs('./datasets', exist_ok=True) # создадим папку для сохранения

tr_mcc_codes.to_csv('datasets/tr_mcc_codes_featured.csv', index=False)
tr_mcc_codes.head(5)

,mcc_code,mcc_description,mcc_full_description,mcc_group
0,742,Ветеринарные услуги (742),Ветеринарные услуги,74
1,1711,Генеральные подрядчики по вентиляции (1711),"Генеральные подрядчики по вентиляции, теплоснабжению, и водопроводу",17
2,1731,Подрядчики по электричеству (1731),Подрядчики по электричеству,17
3,1799,Подрядчики (1799),"Подрядчики, специализированная торговля — нигде более не классифицированные",17
4,2741,Разнообразные издательства (2741),Разнообразные издательства/печатное дело,27


Справочник MCC готов

### Справочник типов категорий

In [4]:
tr_types = pd.read_csv('kaggle_datasets/tr_types.csv', delimiter=';')
tr_types.sample(7, random_state=7)

,tr_type,tr_description
90,3810,н/д
76,8140,Операции по утраченной карте
37,6210,Возврат покупки.POS Зарубеж. ТУ
11,4040,Плата за взнос наличных через POS (в своем тер.банке)
70,8001,Установление расх. лимита по
146,2432,Перевод с карты на счет физ.лица в другом банке через Сбербанк ОнЛ@йн
118,2330,Списание с карты по операции “перевода с карты на карту” через АТМ (в пределах одного тер.банка)



Номер типа операции является внутренним справочником Сбера. Не будем его использовать. 

В описании типа транзакции находится множество информации. Достанем из описания ключевые параметра типа транзакции:
- **устройство**, в котором она была произведена (банкомат, интернет-банк, смс-банк или филиал)
- является ли транзакция операцией **с наличными**, комиссией за использование наличных или безналичная операция
- **регион** операции (зарубежная или российская операция) и банк (домашний банк или "роуминг")
- собственно **тип операции** исходя из названия

In [5]:
device_type_map = {
    'АТМ'                                       : 'АТМ',
    'Банкомат'                                  : 'ATM',
    'через УС'                                  : 'ATM',
    'POS'                                       : 'POS',
    'ТУ'                                        : 'POS',
    'через Мобильный банк'                      : 'SMS',
    'посредством мобильной связи'               : 'mobile',
    'ВСП'                                       : 'VSP',
    'в подразд'                                 : 'VSP',
    'Сбербанк ОнЛ@йн'                           : 'SBOL',
    'через Интернет'                            : 'SBOL',
    'автомат'                                   : 'auto',
}


cash_type_map = {
    'Плата за получение наличных'               : 'cash_trx_payment',
    'Плата за взнос наличных'                   : 'cash_trx_payment',
    'экстренной выдачи наличных'                : 'cash_trx_payment',
    'Взнос наличных'                            : 'cash_trx',
    'Выдача наличных'                           : 'cash_trx',
}


region_map = {
    'Зарубеж'                                   : 'foreign_trx',
    'Россия'                                    : 'other_bank_trx',
    'в чужом АТМ'                               : 'other_bank_trx',
    'из одного ТБ в другой ТБ'                  : 'sberbank_trx',
    'СБ РФ'                                     : 'sberbank_trx',
    'АТМ Сбербанка'                             : 'sberbank_trx',
    'Сбербанк'                                  : 'home_terbank_trx',
    'в своем тер.банке'                         : 'home_terbank_trx',
    'в пределах одного тер.банка'               : 'home_terbank_trx',
    'в своем ТБ'                                : 'home_terbank_trx',
    'в пределах одного филиала'                 : 'home_terbank_trx',
    'в других ТБ'                               : 'other_terbank_trx',
    'в другом тер.банк'                         : 'other_terbank_trx',
}


trx_type_map = {
    'Возврат'                                   : 'Возврат',
    'Перевод'                                   : 'B2C перевод',
    'перевод'                                   : 'B2C перевод',
    'Плата за'                                  : 'Комиссия',
    'Взнос наличных'                            : 'Взнос наличных',
    'Оплата услуг банка'                        : 'Комиссия',
    'Покупка'                                   : 'POS покупка',
    'Пополнение счета'                          : 'Пополнение',
    'Платеж с карты '                           : 'Платеж',
    'Оплата услуги'                             : 'Платеж',
    'Выдача наличных'                           : 'Выдача наличных',
    'Погашение кредита'                         : 'Погашение кредита',
    'Наличные'                                  : 'Выдача наличных',
    'Списание'                                  : 'Списание',
    'списание'                                  : 'Списание',
    'Списание с карты по операции “перевода'    : 'B2C перевод',
    'Комиссия'                                  : 'Комиссия',
    'пополнения зарплатных '                    : 'Заработная плата',
    'перечисления процентов'                    : 'Начисление процентов',
}




def dict_enrichment(dict, field, default_val):
    """
    Проверяет в каждом поле справочника наличие значения `text` и
     подставляет в поле `field` значение из маппера `dict`
    """

    tr_types[field] = pd.Series().astype('object')
    # вызвано https://pandas.pydata.org/docs/whatsnew/v2.1.0.html#deprecated-silent-upcasting-in-setitem-like-series-operations

    for text in dict:
        tr_types.loc[tr_types['tr_description'].str.contains(text), field] = dict[text]
    tr_types[field] = tr_types[field].fillna(default_val)


dict_enrichment(device_type_map,    'device',           'unknown')
dict_enrichment(cash_type_map,      'cash_relation',    'unrelated')
dict_enrichment(region_map,         'region',           'unrelated')
dict_enrichment(trx_type_map,       'type',             'other')

tr_types.to_csv('datasets/tr_types_featured.csv', index=False)
tr_types.sample(5, random_state=11)

# кстати, эту обработку можно было бы переработать, сделав мапперы со структурой, типа:
# 
# trx_type_map = {
#     field : 'region', 
#     maps: {
#         'home_terbank_trx': [
#             'Сбербанк',
#             'в своем тер.банке',
#             'в пределах одного тер.банка',
#             'в своем ТБ',
#             'в пределах одного филиала'
#         ]
#     }
# }
# 
# Делать мы этого, конечно, не будем.
# 
# Потому что читаемость сейчас низкая, но достаточная и далее переиспользуемости этого маппинга не будет. 

,tr_type,tr_description,device,cash_relation,region,type
119,2331,Списание с карты по операции “перевода с карты на карту” через АТМ (из одного ТБ в другой ТБ),АТМ,unrelated,sberbank_trx,B2C перевод
137,2412,Перевод с карты на счет др.лица в одном тер. банке через Сбербанк ОнЛ@йн,SBOL,unrelated,home_terbank_trx,B2C перевод
126,2352,Погашение кредита с карты через АТМ,АТМ,unrelated,unrelated,Погашение кредита
65,7090,Межфилиальные пополнения зарплатных карт,unknown,unrelated,unrelated,Заработная плата
88,2440,н/д,unknown,unrelated,unrelated,other


Справочники готовы. Приступаем к слиянию данных в общую таблицу.

## Формирование общего датасета транзакций

### Слияние данных

In [6]:
# открываем файлы
transactions = pd.read_csv('kaggle_datasets/transactions.csv.zip', delimiter=',')
gender_train = pd.read_csv('kaggle_datasets/gender_train.csv', delimiter=',')

# сливаем файлы
rawset = pd.merge(transactions, gender_train, on='customer_id', how='left')
rawset = pd.merge(rawset, tr_types, on='tr_type', how='left')
rawset = pd.merge(rawset, tr_mcc_codes, on='mcc_code', how='left')

In [7]:
rawset.sample(5, random_state=11)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,device,cash_relation,region,type,mcc_description,mcc_full_description,mcc_group
4852146,24634142,409 16:39:16,5411,1110,-9515.95,25011898,NaN,Покупка. POS ТУ Россия,POS,unrelated,other_bank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54
3368757,4891744,372 19:00:02,5411,1010,-875.91,323492,NaN,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54
6191439,29894123,411 11:01:26,6010,7070,49499.98,888987,NaN,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60
4518851,41783890,453 16:26:28,6010,7070,33688.74,888901,0.0,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60
5414347,56680038,344 09:25:06,5411,1010,-19512.52,689652,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54


Для подготовки сета необходимо дополнительно обработать поля времени операций (tr_datetime) и размера (amount).

Размер операции будет удобно перевести в абсолютные значения с пометкой - расход это или доход. Это позволит работать с валовыми значениями объемов операций вне зависимости от их направления.

Значение tr_datetime необходимо привести к формату даты-времени, желательно с корректной привязкой к временной шкале.

### Обработка поля `amount`

In [8]:
# Добавим абсолютное значение поля как `amount_abs`:
rawset['amount_abs'] = rawset['amount'].apply(abs)

# Добавим направление транзакции, введя поле is_income. Если это входящая тразнакция - ставим 1, если исходящая (расход) - 0.
rawset.loc[rawset['amount']>0, 'is_income'] = 1
rawset.loc[rawset['amount']<0, 'is_income'] = 0

In [9]:
rawset.sample(5, random_state=11)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,device,cash_relation,region,type,mcc_description,mcc_full_description,mcc_group,amount_abs,is_income
4852146,24634142,409 16:39:16,5411,1110,-9515.95,25011898,NaN,Покупка. POS ТУ Россия,POS,unrelated,other_bank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,9515.95,0.0
3368757,4891744,372 19:00:02,5411,1010,-875.91,323492,NaN,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,875.91,0.0
6191439,29894123,411 11:01:26,6010,7070,49499.98,888987,NaN,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,49499.98,1.0
4518851,41783890,453 16:26:28,6010,7070,33688.74,888901,0.0,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,33688.74,1.0
5414347,56680038,344 09:25:06,5411,1010,-19512.52,689652,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,19512.52,0.0


Теперь у нас есть поля amount_abs и is_income. Это позволит нам удобно собирать общие агрегаты по операциям клиента.

### Обработка поля `tr_datetime`

#### Приведение к формату даты-времени

In [10]:
# Приводим значения поля `tr_datetime` к формату даты-времени. Пишем функцию и пробуем ее запустить.

import datetime
def timeformatter(rawdate):
    # разделяем поле на два значения (разделитель - пробел)
    rows = rawdate.split(' ')
    
    # первое значение преобразуем в дату, второе - в формат времени
    days = datetime.timedelta(days=int(rows[0]))
    time = datetime.datetime.strptime(rows[1], '%H:%M:%S')

    return (days+time)


try:
    rawset['tr_datetime'] = rawset['tr_datetime'].apply(timeformatter)

except ValueError as err:
    print('Ошибка: ', err)

Ошибка:  second must be in 0..59


Судя по ошибке, поле содержит данные со значением секунд в них больше 59. Проверим это

In [11]:
# Функция выдает ошибку, потому что почти 100 000 записей имеют количество секунд больше 59

rawset[rawset['tr_datetime'].apply(lambda x: int(x[-2::]) > 59)]

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,device,cash_relation,region,type,mcc_description,mcc_full_description,mcc_group,amount_abs,is_income
77,39026145,43 06:27:60,5499,1010,-1100.50,NaN,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Различные продовольственные магазины (5499),"Различные продовольственные магазины — рынки, магазины со спец-ассортиментом, продажа полуфабрикатов, фирменных блюд, продажа с помощью торговых автоматов",54,1100.50,0.0
221,39026145,123 07:16:60,5499,1010,-4379.54,NaN,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Различные продовольственные магазины (5499),"Различные продовольственные магазины — рынки, магазины со спец-ассортиментом, продажа полуфабрикатов, фирменных блюд, продажа с помощью торговых автоматов",54,4379.54,0.0
233,39026145,132 13:24:60,6010,7020,112295.79,NaN,1.0,Взнос наличных через POS,POS,cash_trx,unrelated,Взнос наличных,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,112295.79,1.0
292,39026145,154 11:02:60,5499,1010,-3593.47,NaN,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Различные продовольственные магазины (5499),"Различные продовольственные магазины — рынки, магазины со спец-ассортиментом, продажа полуфабрикатов, фирменных блюд, продажа с помощью торговых автоматов",54,3593.47,0.0
302,39026145,157 11:55:60,5499,1010,-3357.64,NaN,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Различные продовольственные магазины (5499),"Различные продовольственные магазины — рынки, магазины со спец-ассортиментом, продажа полуфабрикатов, фирменных блюд, продажа с помощью торговых автоматов",54,3357.64,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6848953,61870738,115 11:44:60,5399,1110,-1176.86,NaN,0.0,Покупка. POS ТУ Россия,POS,unrelated,other_bank_trx,POS покупка,Различные товары общего назначения (5399),Различные товары общего назначения,53,1176.86,0.0
6848980,61870738,143 07:00:60,6011,2010,-44918.32,NaN,0.0,Выдача наличных в АТМ Сбербанк России,АТМ,cash_trx,home_terbank_trx,Выдача наличных,Финансовые институты (6011),Финансовые институты — снятие наличности автоматически,60,44918.32,0.0
6849179,61870738,359 16:47:60,5411,1010,-12893.80,496789,0.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,12893.80,0.0
6849257,61870738,420 19:06:60,4829,2370,-89207.77,888905,0.0,Списание с карты на карту по операции <перевода с карты на карту> через Мобильный банк (без комиссии),SMS,unrelated,unrelated,Списание,Денежные переводы (4829),Денежные переводы,48,89207.77,0.0


Таких значений почти 100 000. Судя по тому, что визуально они все равны 60, это ошибка округления. 

Приведем значение 60 секунд к 59 и повторим обработку.

In [12]:
# Исправляем секунды: приводим 60 секунд к значению 59, проверяем есть ли записи со значением секунд больше 59.

def seconds_fixer(string):
    first_part = string[:-2:] # выделяем все до секунд
    last_2chrs = string[-2::] # выделяем секунды

    if int(last_2chrs) > 59:
        last_2chrs = '59'
    
    return (first_part+last_2chrs)

rawset['tr_datetime'] = rawset['tr_datetime'].apply(seconds_fixer)


# Снова пробуем преобразовать строку в формат даты-времени
rawset['tr_datetime'] = rawset['tr_datetime'].apply(timeformatter)


# добавляем отдельно поле даты
rawset['tr_date'] = rawset['tr_datetime'].apply(lambda x: x.date())


In [13]:
rawset.sample(5, random_state=11)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,device,cash_relation,region,type,mcc_description,mcc_full_description,mcc_group,amount_abs,is_income,tr_date
4852146,24634142,1901-02-14 16:39:16,5411,1110,-9515.95,25011898,NaN,Покупка. POS ТУ Россия,POS,unrelated,other_bank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,9515.95,0.0,1901-02-14
3368757,4891744,1901-01-08 19:00:02,5411,1010,-875.91,323492,NaN,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,875.91,0.0,1901-01-08
6191439,29894123,1901-02-16 11:01:26,6010,7070,49499.98,888987,NaN,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,49499.98,1.0,1901-02-16
4518851,41783890,1901-03-30 16:26:28,6010,7070,33688.74,888901,0.0,Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,33688.74,1.0,1901-03-30
5414347,56680038,1900-12-11 09:25:06,5411,1010,-19512.52,689652,1.0,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,19512.52,0.0,1900-12-11


Мы получили значения `tr_datetime` в корректном формате даты-времени и выделили дату (`tr_date`) для большего удобства.

Попробуем найти истинные значения дат.

#### Привязка к временным координатам